<a href="https://colab.research.google.com/github/Jordano-Jackson/MLStudy/blob/main/GAN/GAN/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [2]:
latent_dim = 100

# generator class define
class Generator(nn.Module) :
  def __init__(self) :
    super(Generator, self).__init__();

    def block(input_dim, output_dim, normalize=True) :
      layers = [nn.Linear(input_dim,output_dim)]
      if(normalize) :
        # batch normalization
        layers.append(nn.BatchNorm1d(output_dim, 0.8))
      layers.append(nn.LeakyReLU(0.2, inplace=True));
      return layers
    
    self.model = nn.Sequential(
        *block(latent_dim, 128, normalize=False),
        *block(128,256),
        *block(256,512),
        *block(512,1024),
        nn.Linear(1024, 1*28*28),
        nn.Tanh()
    )

  def forward (self, z) :
    img = self.model(z)
    img = img.view(img.size(0), 1,28,28)
    return img

In [3]:
# discriminator class define
class Discriminator(nn.Module) :
  def __init__(self) :
    super(Discriminator, self).__init__()

    self.model = nn.Sequential(
        nn.Linear(1*28*28, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512,256),
        nn.LieakyReLU(0.2, inplace=True),
        nn.Linear(256,1),
        nn.Sigmoid(),
    )

  # do discrimination on image
  def forward(self, img) :
    flattened = img.view(img.size(0),-1)
    output = self.model(flattened);

    return output

In [4]:
transforms_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_dataset = datasets.MNIST(root="./dataset", train=True, transform=transforms_train)
dataloader = torch.utils.data.Dataloader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

RuntimeError: ignored